In [61]:
import numpy as np
import math
import random
import pprint
import pygame

In [6]:
#===========CHANGABLE PARAMETERS=================
REWARD     = 1
PENALTY    = -1
DISCOUNT   = 0.9
LEARN_RATE = 0.1
def get_explore_rate(epoch):
    return max(0.1,1-math.log(epoch)/10)

#===========DEFINE CONSTANTS AND DICS=============
WALL_LEN = 1
PADDLE_H = 0.2

init_state = (0.5, 0.5, 0.03, 0.01, 0.5 - PADDLE_H / 2)

ACTION_DIC={0:-0.04, #'UP'
            1:0,     #'STAY'
            2:0.04}  #'DOWN'

BOARD_SIZE = 12
X_VBALL_DIS = [-1,1]
Y_VBALL_DIS = [-1,0,1]

PADDLE_SPACE = 12
PADDLE_X     = 1

STATE_SPACE = (BOARD_SIZE,BOARD_SIZE,len(X_VBALL_DIS),len(Y_VBALL_DIS),PADDLE_SPACE)

X_V_TSH = 0.03
Y_V_TSH = 0.015

In [7]:
#==============DEFINE STATE CLASS===============
class state:
    
    def __init__(self,ball_x,ball_y,velocity_x,velocity_y,paddle_y,reward,end_state = 0):
        self.ball_x = ball_x              #real numbers on the interval [0,1]
        self.ball_y = ball_y
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y
        self.paddle_y = paddle_y
        self.reward = reward
        self.state_tuple = (ball_x,ball_y,velocity_x,velocity_y,paddle_y)
        self.end_state = end_state
        self._extract()
        
    def _extract(self):
        self.x_grid = min(math.floor(11*self.ball_x),BOARD_SIZE-1)
        self.y_grid = min(math.floor(11*self.ball_y),BOARD_SIZE-1)
        if(self.velocity_x>0): 
            self.x_v_sign = 1
        else: 
            self.x_v_sign = -1
            
        if(self.velocity_y>0):
            self.y_v_sign = 1
        else: 
            self.y_v_sign = -1
        
        self.paddle_grid = min(math.floor(11 * self.paddle_y / (1 - PADDLE_H)),PADDLE_SPACE-1)
        self.space_tuple = (self.x_grid,self.y_grid,self.x_v_sign,self.y_v_sign,self.paddle_grid)

In [8]:
#=======DEFINE MORE HELPER FUNCTIONS===========
def bounce(cur_state,action):
#     print('ENTER!!!!')
    n_ball_v_x = cur_state.velocity_x
    n_ball_v_y = cur_state.velocity_y
    
    n_ball_x = cur_state.ball_x + n_ball_v_x
    n_ball_y = cur_state.ball_y + n_ball_v_y

    #======paddle should in range(0,0.8)
    n_paddle_y = min(WALL_LEN-PADDLE_H,max(0,cur_state.paddle_y + action))
    
    c_reward = 0
    
    #==the ball is off the top of the screen==
    if n_ball_y < 0:
#         print('1111111111111111111111111')
        n_ball_y = -n_ball_y
        n_ball_v_y = -n_ball_v_y
        
    #==the ball is off the bottom of the screen==
    if n_ball_y > 1:
#         print('22222222222222222222222222')
        n_ball_y = 2*WALL_LEN-n_ball_y
        n_ball_v_y = -n_ball_v_y
        
    #==the ball is off the left edge of the screen==
    if n_ball_x < 0:
#         print('33333333333333333333333333')
        n_ball_x = -n_ball_x
        n_ball_v_x = -n_ball_v_x
        
    #== the ball bouncing off the paddle==
#     print('n_paddle_y+PADDLE_H',n_paddle_y+PADDLE_H,'n_ball_y', n_ball_y,'n_paddle_y', n_paddle_y)
#     print(n_paddle_y+PADDLE_H <= n_ball_y <= n_paddle_y)
#     print(n_ball_x >= 1)
    if n_ball_x >= 1 and (n_paddle_y+PADDLE_H >= n_ball_y >= n_paddle_y):
#         print('yuayyyaysaysdyuasfhiuadsfhiua!')
        n_ball_x = 2*PADDLE_X - n_ball_x
        n_ball_v_x, n_ball_v_y = update_speed_rand(n_ball_v_x,n_ball_v_y)
        c_reward = REWARD
    elif n_ball_x > 1:
#         print('444444444444444444444444444')
        c_reward = PENALTY
    return n_ball_x,n_ball_y,n_ball_v_x,n_ball_v_y,n_paddle_y,c_reward

def update_speed_rand(velocity_x,velocity_y):
    vx_delta = random.choice(range(-15,16,1))/1000
    vy_delta = random.choice(range(-3,4,1))  /100
    sign_x = velocity_x/abs(velocity_x)
    sign_y = velocity_y/abs(velocity_y)
    n_ball_v_x = sign_x*max(X_V_TSH,abs(vx_delta+velocity_x))
    n_ball_v_y = sign_y*max(Y_V_TSH,abs(vy_delta+velocity_y))
    return n_ball_v_x, n_ball_v_y

def check_termination(cur_state):
    pad_rang = range(cur_state.paddle_y-PADDLE_H,cur_state.paddle_y)
    if cur_state.ball_x>=WALL_LEN and cur_state.ball_y not in pad_rang:
        return True
    else:
        return False
        
def proceed_one_step(cur_state,action):

    n_ball_x,n_ball_y,n_ball_v_x,n_ball_v_y,n_paddle_y,c_reward = bounce(cur_state,action)
    end = 0
    if c_reward == -1:
        end = 1
#     print(n_ball_x,n_ball_y,n_ball_v_x,n_ball_v_y,n_paddle_y,c_reward,end,action)
    n_state = state(n_ball_x,n_ball_y,n_ball_v_x,n_ball_v_y,n_paddle_y,c_reward,end_state=end)
    return n_state

In [9]:
#=============DEFINE Q-AGENT CLASS==============
class q_agent:
    
    def __init__(self):
        self.q_table = np.zeros(STATE_SPACE+(len(ACTION_DIC),)) 
        self.end_state = 0
        
    def set_table(self,loc,val):
        self.q_table[loc] = val
    
    def get_table(self,loc):
#         print(loc)
        return self.q_table[loc]
    
    def get_act(self,cur_state,i,mode = 'train'):
        if mode=='train' and random.random()<get_explore_rate(i):
            return random.choice(range(0,3,1))
        return np.argmax(self.get_table(cur_state.space_tuple))
    

In [10]:
#=============DEFINE TRAIN FUCNTION=============
#init_state = (0.5, 0.5, 0.03, 0.01, 0.5 - PADDLE_H / 2)
#self.space_tuple = (self.x_grid,self.y_grid,self.x_v_sign,self.y_v_sign,self.paddle_grid)
def train(epoch_num,q_ag):

    tot_bounce = 0
    for i in range(1,epoch_num+1):
#         print('i=',i)
        temp_bounce = 0
        cur_state = state(0.5, 0.5, 0.03, 0.01, 0.5 - PADDLE_H / 2,0)
        while True:
            action = q_ag.get_act(cur_state,i)
#             action = 1
#             print('act=',action)
            n_state = proceed_one_step(cur_state,ACTION_DIC[action])
            old_val = q_ag.get_table(cur_state.space_tuple+(action,))


            prd_max = np.max(q_ag.get_table(n_state.space_tuple))
            prd_tuple = (n_state.x_grid,n_state.y_grid,n_state.x_v_sign,n_state.y_v_sign,n_state.paddle_grid,prd_max)

            new_val = (1-LEARN_RATE)*old_val + LEARN_RATE*(n_state.reward + DISCOUNT*prd_max)
            q_ag.set_table(cur_state.space_tuple+(action,),new_val)
#             print('22222222222=',prd_max)
#             print('33333333333333=',n_state.space_tuple+(action,))
#             print('44444444444=',q_ag.get_table(n_state.space_tuple+(action,)))
            if n_state.end_state == 1:
                break
            if n_state.reward == REWARD:
                temp_bounce+=1
#                 print('this round=',temp_bounce)
            cur_state = n_state
        tot_bounce+=temp_bounce
#         print()
#         print()
        if i%1000 == 0:
            print('loop',i)
            print(q_ag.q_table.sum())
            print('now average bounce is', tot_bounce/i)
            print('bounce=',tot_bounce)
            print()
    

print(n_ball_x,n_ball_y,n_ball_v_x,n_ball_v_y,n_paddle_y,c_reward,end)

In [11]:
q_ag = q_agent()
train(100000,q_ag)

loop 1000
318.004188741
now average bounce is 7.584
bounce= 7584

loop 2000
428.092858912
now average bounce is 9.1875
bounce= 18375

loop 3000
486.47267232
now average bounce is 9.879666666666667
bounce= 29639

loop 4000
549.778133749
now average bounce is 10.557
bounce= 42228

loop 5000
575.992139196
now average bounce is 10.9886
bounce= 54943

loop 6000
608.690379091
now average bounce is 11.479666666666667
bounce= 68878

loop 7000
659.37394784
now average bounce is 11.857
bounce= 82999

loop 8000
669.477760566
now average bounce is 12.1935
bounce= 97548

loop 9000
688.413142071
now average bounce is 12.444666666666667
bounce= 112002

loop 10000
708.855422405
now average bounce is 12.6357
bounce= 126357

loop 11000
719.254577433
now average bounce is 12.817454545454545
bounce= 140992

loop 12000
716.409325357
now average bounce is 12.948916666666667
bounce= 155387

loop 13000
732.735614385
now average bounce is 13.065846153846154
bounce= 169856

loop 14000
742.75309418
now average b

In [21]:
def test(epoch_num,q_ag):
    tot_bounce = 0
    for i in range(epoch_num):
        temp_bounce = 0
        cur_state = state(0.5, 0.5, 0.03, 0.01, 0.5 - PADDLE_H / 2,0)
        while True:
            action = q_ag.get_act(cur_state,i,mode='hei')
            n_state = proceed_one_step(cur_state,ACTION_DIC[action])
            if n_state.end_state == 1:
                break
            if n_state.reward == REWARD:
                temp_bounce+=1
            cur_state = n_state
        tot_bounce+=temp_bounce
#         print('loop',i)
#         print('now average bounce is', tot_bounce/(i+1))
#         print('bounce in this game=',temp_bounce)
#         print()
    print('the avg bounce =',tot_bounce/epoch_num)

In [59]:
test(200,q_ag)

the avg bounce = 19.065


In [ ]:
q_ = q_agent()
train(100000,q_)

In [63]:
pygame.init()

size = width, height = 320, 240
speed = [2, 2]
black = 0, 0, 0

screen = pygame.display.set_mode(size)

ball = pygame.image.load("ball.bmp")
ballrect = ball.get_rect()

while 1:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: sys.exit()

    ballrect = ballrect.move(speed)
    if ballrect.left < 0 or ballrect.right > width:
        speed[0] = -speed[0]
    if ballrect.top < 0 or ballrect.bottom > height:
        speed[1] = -speed[1]

    screen.fill(black)
    screen.blit(ball, ballrect)
    pygame.display.flip()

error: Couldn't open ball.bmp